In [1]:
import secrets

In [2]:
def inverseMod(a, mod):
    return pow(a, -1, mod)

In [23]:
class Curve:
    def __init__(self, a, b, p):
        # curve configuration
        # y^2 = x^3 + a*x + b
        self.a = a
        self.b = b
        self.p = p
    
    def contains(self, x, y):
        return (x**3 + x * self.a + self.b - y**2) % self.p == 0

In [24]:
class Point:
    def __init__(self, x, y, curve):
        xp = x % curve.p
        yp = y % curve.p
            
        if not curve.contains(xp, yp):
            raise Exception("curve does not contain point")
        
        self.x = xp
        self.y = yp
        self.curve = curve
        
    def copy(self):
        return Point(self.x, self.y, self.curve)
    
    def __neg__(self):
        return Point(self.x, -self.y, self.curve)
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y and self.curve == other.curve
        
    def __add__(self, other):
        if other.curve != self.curve:
            raise Exception("Can not add with different curves")
        
        if self == other:
            beta = (3 * self.x * self.x + self.curve.a) * inverseMod(2 * self.y, self.curve.p)
        else:
            beta = (other.y - self.y) * inverseMod(other.x - self.x, self.curve.p)

        xnew = beta * beta - self.x - other.x
        ynew = beta * (self.x - xnew) - self.y

        return Point(xnew, ynew, self.curve)
    
    def __sub__(self, other):
        return self + (-other)
    
    def __mul__(self, k):
        temp = self.copy()
        kAsBinary = bin(k)
        kAsBinary = kAsBinary[2:len(kAsBinary)]

        for i in range(1, len(kAsBinary)):
            temp = temp + temp
            if kAsBinary[i: i+1] == '1':
                temp = temp + self

        return temp
    
    def __rmul__(self, k):
        return self * k

In [25]:
curve = Curve(a=1, b=1, p=23)
g = Point(x=7, y=11, curve=curve)

## ECDH

In [26]:
# Alice gera publica
alicePrivate = secrets.randbelow(g.curve.p)
alicePublic = alicePrivate * g
print(f'{ alicePublic.x = }')

 alicePublic.x = 13


In [27]:
# Bob gera publica
bobPrivate = secrets.randbelow(g.curve.p)
bobPublic = bobPrivate * g
print(f'{ bobPublic.x = }')

 bobPublic.x = 5


In [28]:
# Alice computa shared a partir de publica de Bob
aliceShared = alicePrivate * bobPublic
print(f'{ aliceShared.x = }')

 aliceShared.x = 13


In [29]:
# Bob computa shared a partir de publica de Alice
bobShared = bobPrivate * alicePublic
print(f'{ bobShared.x = }')

 bobShared.x = 13


## Proposed

### Registration (gerar chaves publicas)

In [30]:
# Alice gera publica
alicePrivate = secrets.randbelow(g.curve.p)
alicePublic = alicePrivate * g
print(f'{ alicePublic.x = }')

 alicePublic.x = 17


In [31]:
# Bob gera publica
bobPrivate = secrets.randbelow(g.curve.p)
bobPublic = bobPrivate * g
print(f'{ bobPublic.x = }')

 bobPublic.x = 13


### Computacao (segredo da sessao e envio)

In [33]:
aliceSessionSecret = secrets.randbelow(g.curve.p)
toBob = (alicePrivate + aliceSessionSecret) * bobPublic
print(f'{ toBob.x = }')

 toBob.x = 13


In [35]:
bobSessionSecret = secrets.randbelow(g.curve.p)
toAlice = (bobPrivate + bobSessionSecret) * alicePublic
print(f'{ toAlice.x = }')

 toAlice.x = 5


### Computacao chave sessao

In [36]:
aliceSharedPartial = inverseMod(alicePrivate, g.curve.p) * toAlice - alicePublic
print(f'{ aliceSharedPartial.x = }')

aliceShared = aliceSharedPartial + (aliceSessionSecret * g)
print(f'{ aliceShared.x = }')

 aliceSharedPartial.x = 13
 aliceShared.x = 17


In [37]:
bobSharedPartial = inverseMod(bobPrivate, g.curve.p) * toBob - bobPublic
print(f'{ bobSharedPartial.x = }')

bobShared = bobSharedPartial + (bobSessionSecret * g)
print(f'{ bobShared.x = }')

ValueError: base is not invertible for the given modulus